In [1]:
from datetime import datetime as dt  #to manipulate dates

import numpy as np  #to cleanse data
import openpyxl
import pandas as pd
import plotly.express as px  #to create interactive charts
import plotly.graph_objects as go  #to create interactive charts
from datetime import datetime

In [3]:
dtype_dict = {
    'Description': 'string',
    'Amount': 'float',
    'Type': 'string',
    'Current balance': 'float',
    'Status': 'string'
}

ytd_data = pd.read_csv(
    'D:\\Sean\\Documents\\Personal\\Budget\\YTD_downloads\\2025\\SOFI-Checking.csv', dtype=dtype_dict, parse_dates=['Date']
)


ytd_data['Account'] = 'sofi_checking'
ytd_data.rename(columns={'Date':'Trans. Date','Current balance':'CumAmount'}, inplace=True)

ytd_data['Date'] = pd.to_datetime(ytd_data['Trans. Date'].dt.strftime('%m/%Y'), format='%m/%Y')

In [4]:
ytd_data

,Trans. Date,Description,Type,Amount,CumAmount,Status,Account,Date
0,2025-10-24,Roundup *24 HOUR FITNESS - 475,Roundup,-0.07,992.07,Posted,sofi_checking,2025-10-01
1,2025-10-24,24 HOUR FITNESS - 475,Debit Card,-4.93,992.14,Posted,sofi_checking,2025-10-01
2,2025-10-23,DEPT EDUCATION,Direct Payment,-442.73,997.07,Posted,sofi_checking,2025-10-01
3,2025-10-21,From Savings - 3552,Deposit,1000.00,1439.80,Posted,sofi_checking,2025-10-01
4,2025-10-20,WELLS FARGO BANK NA,Withdrawal,-1000.00,439.80,Posted,sofi_checking,2025-10-01
...,...,...,...,...,...,...,...,...
258,2025-01-12,To Savings - 3552,Withdrawal,-1000.00,1619.69,Posted,sofi_checking,2025-01-01
259,2025-01-11,Zelle® Payment from Don Lewis,Direct Payment,967.00,2619.69,Posted,sofi_checking,2025-01-01
260,2025-01-08,VENMO,Direct Payment,-1633.74,1652.69,Posted,sofi_checking,2025-01-01
261,2025-01-07,From Savings - 3552,Deposit,1634.00,3286.43,Posted,sofi_checking,2025-01-01


In [45]:
# Discover Transactions
mapping = {
    # Direct Deposit
    'interest': ['INTEREST'],
    'transfer' :['TRANSFER','From Savings - 3552','Discover','WELLS FARGO BANK','Venmo','Sean Lewis'],
    'laptop': ['BEST BUY'],
    'student_loan': ['DEPT EDUCATION'],
    'state_farm': ['STATE FARM'],
    'clothing': ['SP RAISED BY SOCIETY'],
    'Jayelin':['Jayelin'],
    'care_credit':['Synchrony'],
    'Don':['Credit One', 'Don Lewis'],
    'Tahjei': ['Peak Living', 'Tahjei'],
    'Sharon': ['Sharon'],
    'Simone': [r'SSP\*TNTGYMNASTICSANDCH'],
    'Ryan': [r'CASH APP\*RYAN TITE'],
    'Donesha': [r'CASH APP\*CAIRO DANIEL'],
    'Kenny': ['Zelle® Payment from Kenny Thai'],
    'Roundup': ['Roundup'],
    'parking': [r'CITY OF SM PARKPAYBYPH'],
    'fast_food': ['Subway','LIFE CAFE',r'SQ \*SHAKE SHACK',r'DD \*DOORDASH GREEKSTRE','SUPERB SUNNY PTY LTD',r'DD \*SUSHIKUU'],
    'groceries': ['Safeway','WHOLEFDS SMT 10150'],
    'everything_store': ['Target'],
    'misc':['AppFolio, Inc.'],
    'Sydney': ['JALU FRAGRANCES','MCDONALDS BURWOOD','WOOLWORTHS','TransportforNSW','CHATSWOOD','MUBASSHIR GHULAM MOHAM','WOMBEYAN','GOULBURN','Kopi Kenangan Westfiel','Back row west','BANKSTOWN','MOSHTIX PTY LTD','OFFICEWORKS 0251','SYDNEY','Syd',r'SQ \*BEET - COFFEE  FOO',r'SQ \*BLOOMING CAFE','JB HI FI', 'BIG W','TheGroundsCoffeeFact',
               r'SQ \*MERCH FAN',r'SQ \*FISHBOWL SOUTH EVE','ROMEOS FOODHALL SOUTH','PEPPER LUNCH','LS GAMES CHAIN',r'ZLR\*The Cheesecake Sho',r'DD \*DOORDASH TIDAPERSI',
               'SUMMER VIBE PTY LTD','ALAM ENJOY PTY LTD',r'SQ \*CENTENNIAL VINEYAR','THE TRUSTEE FOR OZKII','Kafe Kooks','The Naked Duck Bond','Brooklyn Donuts Chatsw',
               'BABYLON RESTAURANT A','IMMMG PTY LTD','ALDI STORES','LS Dose Espresso',r'SQ \*ENVOY CAFE','CROSSLAB PTY LTD','GOLDEN TREE HOLDINGSGO','SAKURA MARKET',
               r'SQ \*BUTTERCRUMBS CROIS',r'SQ \*ICE KIRIN BAR BURW','SMIGGLE PTY LTD',r'ZLR\*Monsterthreads New','MUMU LIFE',r'SQ \*BALLSY TEA',r'kikki\.K',
               'KINGPIN BOWLING','HEALTHY CARE BURWOOD P','CCW MUSEUM',r'SQ \*THE TEA COLLECTIVE','ART ON KING','COLES 0852'],
    'transfer': ['To Savings'],
    'parking': ['CITY OF SM PW PARKING'],
    'gas_station': ['ARCO*'],
    'taxes': ['IRS'],
    'atm': [r'BMO BANK N\.A\.'],
    'London': ['UKVI ETAMOB10000702110'],
    'conv_store': ['CVS/PHARM'],
    'learning': ['SPRINGER NATURE'],
    'gym': ['24 HOUR FITNESS']
}

for k, v in mapping.items():
    ytd_data.loc[ytd_data.Description.str.contains('|'.join(v), case=False), 'Short_Desc'] = k
    ytd_data.loc[ytd_data.Description.str.contains("venmo", case=False) & (
            ytd_data['Amount'] == -816.87), 'Short_Desc'] = 'rent'
    ytd_data.loc[ytd_data.Description.str.contains("venmo", case=False) & (
            ytd_data['Amount'] == -1803.40), 'Short_Desc'] = 'vincent_debt'
    ytd_data.loc[ytd_data.Description.str.contains("venmo", case=False) & (
            ytd_data['Amount'] == -3606.80), 'Short_Desc'] = 'vincent_debt'
    ytd_data.loc[ytd_data.Description.str.contains("venmo", case=False) & (
            ytd_data['Amount'] == -1473.72), 'Short_Desc'] = 'vincent_debt'
    ytd_data.loc[ytd_data.Description.str.contains("venmo", case=False) & (
            ytd_data['Amount'] == -50), 'Short_Desc'] = 'joshua_mitchell'
    ytd_data.loc[ytd_data.Description.str.contains("venmo", case=False) & (
            ytd_data['Amount'] == -30), 'Short_Desc'] = 'huong_nguyen'
    


In [46]:
ytd_data[ytd_data['Short_Desc'].isnull()].sort_values(by='Amount', ascending=False)

,Trans. Date,Description,Type,Amount,CumAmount,Status,Account,Date,Short_Desc


In [47]:
ytd_data.loc[ytd_data['Short_Desc'].isnull(), 'Short_Desc'] = 'unsure'

In [60]:
reocurring_list = ['rent','student_loan','laptop', 'vincent_debt','interest']
ytd_data.loc[ytd_data.Short_Desc.isin(reocurring_list), 'Reoccurring_Flag'] = 'Yes'
ytd_data.loc[~ytd_data.Short_Desc.isin(reocurring_list), 'Reoccurring_Flag'] = 'No'

mapping = {
    'Housing': ['rent'],
    'Transportation': ['parking'],
    'Food': ['groceries','fast_food'],
    'Insurance': ['state_farm'],
    'Utilities': [],
    'Medical': [],
    'Government': ['student_loan','taxes'],
    'Savings': ['income'],
    'Personal_Spending': ['laptop','clothing','learning','gym','conv_store','everything_store'],
    'Recreation_Entertainment': [],
    'Travel': ['Sydney','gas_station','London'],
    'Misc': ['transfer','Roundup','atm'],
    'People': ['vincent_debt','joshua_mitchell','huong_nguyen','Sharon','Tahjei', 'Ryan','Simone','Don','Donesha'],
    'Payment_and_Interest': ['interest','care_credit']
}

for k, v in mapping.items():
    ytd_data.loc[ytd_data.Short_Desc.isin(v), 'Category_2'] = k

ytd_data[ytd_data['Category_2'].isnull()].sort_values(by='Description', ascending=False).sort_values(by='Short_Desc',
                                                                                                     ascending=False)

,Trans. Date,Description,Type,Amount,CumAmount,Status,Account,Date,Short_Desc,Reoccurring_Flag,Category_2


In [61]:
x = ytd_data.groupby(['Reoccurring_Flag']).size().reset_index(name='count')

x.sort_values(by='count', ascending=False)

,Reoccurring_Flag,count
0,No,251
1,Yes,12


In [62]:
ytd_sorted = ytd_data.sort_values(by='Trans. Date').reset_index(drop=True)

# Ensure 'Sort_Date' is also in datetime format for the entire DataFrame
ytd_sorted['Trans. Date'] = pd.to_datetime(ytd_sorted['Trans. Date'])
ytd_sorted['Year'] = ytd_sorted['Trans. Date'].dt.year
ytd_sorted['Counter'] = ytd_sorted.groupby('Short_Desc').cumcount() + 1
ytd_sorted

,Trans. Date,Description,Type,Amount,CumAmount,Status,Account,Date,Short_Desc,Reoccurring_Flag,Category_2,Year,Counter
0,2025-01-01,From Savings - 3552,Deposit,450.00,1652.43,Posted,sofi_checking,2025-01-01,transfer,No,Misc,2025,1
1,2025-01-07,From Savings - 3552,Deposit,1634.00,3286.43,Posted,sofi_checking,2025-01-01,transfer,No,Misc,2025,2
2,2025-01-08,VENMO,Direct Payment,-1633.74,1652.69,Posted,sofi_checking,2025-01-01,transfer,No,Misc,2025,3
3,2025-01-11,Zelle® Payment from Don Lewis,Direct Payment,967.00,2619.69,Posted,sofi_checking,2025-01-01,Don,No,People,2025,1
4,2025-01-12,To Savings - 3552,Withdrawal,-1000.00,1619.69,Posted,sofi_checking,2025-01-01,transfer,No,Misc,2025,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
258,2025-10-20,WELLS FARGO BANK NA,Withdrawal,-1000.00,439.80,Posted,sofi_checking,2025-10-01,transfer,No,Misc,2025,65
259,2025-10-21,From Savings - 3552,Deposit,1000.00,1439.80,Posted,sofi_checking,2025-10-01,transfer,No,Misc,2025,66
260,2025-10-23,DEPT EDUCATION,Direct Payment,-442.73,997.07,Posted,sofi_checking,2025-10-01,student_loan,Yes,Government,2025,2
261,2025-10-24,24 HOUR FITNESS - 475,Debit Card,-4.93,992.14,Posted,sofi_checking,2025-10-01,gym,No,Personal_Spending,2025,1


In [64]:
columns = ['Trans. Date', 'Description', 'Amount', 'Category', 'Short_Desc', 'Reoccurring_Flag',
                     'Category_2', 'Date', 'Year', 'Sort_Date', 'CumAmount', 'Counter', 'Account']
ytd_sorted['Category'] = 'NA'
ytd_sorted['Sort_Date'] = ytd_sorted['Trans. Date']

sofi_checking = ytd_sorted[columns]

In [65]:
sofi_checking.to_excel('D:\\Sean\\Documents\\Personal\\Budget\\2025_output\\sofi_checking_Curated_Bills.xlsx')